In [2]:
import numpy as np
import os, sys, cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import Metashape as PhotoScan
import pandas as pd
from glob import glob
from zipfile import ZipFile
import time
def progress_print(p):
    elapsed = float(time.time() - start_time)
    if p:
        sec = elapsed / p * 100
        print('\rCurrent task progress: {:.2f}%'.format(p), end='')
    else:
        print('\rCurrent task progress: {:.2f}%'.format(p), end='') #if 0% progress
        
global start_time

## userArg
project_name = "wegan_2ha"
web_Title = "2HA"
##

agisoftProjectPath = fr'../docSave/{project_name}.psx'
doc = PhotoScan.Document()
doc.open(agisoftProjectPath)
chunk = doc.chunk
start_time = time.time()
chunk.exportTiledModel(f'../docSave/{project_name}.zip', 
                       format=PhotoScan.TiledModelFormatCesium,
                       texture_format=PhotoScan.ImageFormatJPEG,
                       model_format=PhotoScan.ModelFormatCOLLADA,
                       raster_transform=PhotoScan.RasterTransformNone,
                       progress=progress_print)

# from zipfile import ZipFile
srcZip = f'../docSave/{project_name}.zip'
archiveName = os.path.basename(srcZip).lower().replace(".zip", "")
cesiumDir = "..//Cesium-1.84/"
dstDir = os.path.join(cesiumDir, archiveName)
os.makedirs(dstDir,exist_ok=True) # dev: True

with ZipFile(srcZip, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall(dstDir)

os.remove(srcZip)
# TODO: html name check
baseWebPage = os.path.join(cesiumDir, "base.html")
newWebPage = os.path.join(cesiumDir, f"{archiveName}.html")
newPageUrl = f"http://192.168.0.115:8000/{archiveName}/tileset.json"
with open(baseWebPage, "r") as f_in:
    data = f_in.readlines()
    data[43] = data[43].replace("\'\'", f"\'{newPageUrl}\'")
    data[11] = data[11].replace("Cesium Demo", web_Title)
    with open(newWebPage, "w") as f_out:
        f_out.writelines(data)

Current task progress: 100.00%